In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from case1_datagen import generate_collocation_points, generate_interface_collocation_points
from case1_loss_fns import pde_loss, bc_loss, interface_loss, flexural_rigidity, normalise, denormalise
from case1_beamdoublenet import BeamDoubleNet
from case1_bayesian_opt import run_bayesian_optimisation

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device}")

if device.type == 'cpu':
    torch.set_num_threads(16)
    print(f"Limiting cpu threads to: {torch.get_num_threads()}")

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Assume the following functions are defined elsewhere:
# def generate_collocation_points(n, x_ranges): ...
# def normalise(x, xmin, xmax): ...
# def generate_interface_collocation_points(n, n_interface, x_ranges, interface_x, interface_width): ...


# --- Your Data Generation Parameters ---
n_collocation = 3000
#n_interface = 1000
#x_ranges = [(0,2), (2,3)]
#interface_x = 2.0
#interface_width = 0.1
xmin, xmax = 0.0, 3.0
q0 = 500
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#Generate Uniform Collocation Points for Each Domain

x_phys_pde = np.random.uniform(xmin, xmax, (n_collocation, 1)).astype(np.float32)

# --- Convert to PyTorch Tensor and Finalize ---
# Normalize the points
x_pde_torch = torch.from_numpy(normalise(x_phys_pde, xmin, xmax)).to(torch.float32).to(device)

shuffled_indices = torch.randperm(len(x_pde_torch))
x_pde_torch = x_pde_torch[shuffled_indices]
x_pde_torch.requires_grad_(True)

data = {
    'x_pde': x_pde_torch,  # Single tensor for all PDE points
    'xmin': xmin,
    'xmax': xmax,
    'device': device,
    'q0': q0,
}

print(f"Total PDE points for the single beam: {len(x_pde_torch)}")


# --- VISUALISATION (Simplified) ---
x_pde_combined_flat = x_pde_torch.cpu().detach().numpy().flatten() # Get flat array for plotting
plt.figure(figsize=(8, 2))
plt.scatter(x_pde_combined_flat, np.zeros_like(x_pde_combined_flat), alpha=0.5, label='Single Beam PDE points (normalized)')
# No interface line needed for a single beam
plt.xlabel('Normalized x')
plt.yticks([])
plt.title('Collocation Points Along Single Beam')
plt.legend()
plt.show()


In [3]:
# Bayesian search for the optimal parameters

#epochs_per_trial = 300  # A small number of epochs for the search
#n_trials = 50           # A reasonable number of trials

#print("\n--- Starting Bayesian Optimization Search ---")
#best_params = run_bayesian_optimisation(data, epochs_per_trial, n_trials, device)
#print("\n--- Search Complete. Best Parameters Found ---")
#print(best_params)

In [4]:
# Instantiate the models

n_units=40
n_layers=4
pde_weight = 1.0#best_params['pde_weight']
bc_weight = 1.0#best_params['bc_weight']
#if_weight = best_params['if_weight']
#if_cont_weight = best_params['if_cont_weight']
#if_shear_weight = best_params['if_shear_weight']
lr = 0.01#best_params['learning_rate']


model_single_beam = BeamDoubleNet(
    input_dim=1, output_dim=2,
    n_units=n_units, n_layers=n_layers,
    pde_weight=pde_weight, bc_weight=bc_weight, if_weight=if_weight,
    #if_cont_weight=if_cont_weight, if_shear_weight=if_shear_weight,
).to(device)

#model_beam2 = BeamDoubleNet(
#    input_dim=1, output_dim=2,
#    n_units=n_units, n_layers=n_layers,
#    pde_weight=pde_weight, bc_weight=bc_weight,# if_weight=if_weight,
#    #if_cont_weight=if_cont_weight, if_shear_weight=if_shear_weight,
#).to(device)


# Set up the optimizer (a single optimizer to update the params of BOTH models)

optimizer = torch.optim.Adam(
    list(model_single_beam.parameters()),
    lr=lr
)

In [ ]:
# training loop

losses = []
epochs=500

for ep in range(epochs):
    # This ensures x_pde_torch has requires_grad=True, even in a notebook
    #x1_pde_torch.requires_grad_(True)
    #x2_pde_torch.requires_grad_(True)          REQUIRED WHEN MINIBATCHING
    optimizer.zero_grad()

    loss_residual = pde_loss(model_single_beam, x_pde_torch, xmin, xmax, q0)# + pde_loss(model_beam2, x2_pde_torch, xmin, xmax)
    loss_boundary = bc_loss(model_single_beam, xmin, xmax)
    loss_interface = interface_loss(model_single_beam xmin, xmax), q0# if_shear_weight, if_cont_weight)

    total_loss = pde_weight*loss_residual + bc_weight*loss_boundary #+ if_weight * loss_interface

    #backpropagation and optimisation via pytorch
    total_loss.backward()
    optimizer.step()
    losses.append(total_loss.item())

    if ep % int(epochs/10) == 0:
        print(f"Epoch {ep}: Total Loss {total_loss.item():.4e} | "
              f"PDE {loss_residual.item():.4e} | "
              f"BC {loss_boundary.item():.4e}")
              #f"IF {loss_interface.item():.4e}")
        
plt.figure(figsize=(8, 4))
plt.plot(losses)
plt.yscale('log')
plt.xlabel('Epoch')
plt.ylabel('Total Loss')
plt.title('Training Loss History (Domain Decomposition)')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

# 1. Beam and material properties
E, D = 210e9, 0.05
I = np.pi/64 * D**4
EI = E * I

def w_analytic(x_phys, q0_val, EI_val, L_val):
    """
    Calculates the analytic deflection for a uniformly loaded cantilever beam.
    x_phys: physical x-coordinates (m)
    q0_val: uniformly distributed load (N/m)
    EI_val: flexural rigidity (Nm^2)
    L_val: total beam length (m)
    """
    w = (q0_val / EI_val) * (
        -x_phys**4 / 24.0 + L_val * x_phys**3 / 6.0 - (L_val**2) * x_phys**2 / 4.0
    )
    return w

# Generate x values and normalize
xmin, xmax = 0.0, 3.0
x_phys_plot = np.linspace(xmin, xmax, 500)

# Evaluate analytic solution
w_true = w_analytic(x_phys_plot, q0_val=500, EI_val=EI, L_val=3)

x_norm_plot_torch = torch.from_numpy(normalise(x_phys_plot, xmin, xmax)).reshape(-1, 1).to(torch.float32).to(device)

# Use boolean masks to apply the correct model to each segment



w_pinn = np.zeros_like(x_phys_plot)

# Set models to evaluation mode and get predictions
model_single_beam.eval()
with torch.no_grad():
    w_pinn = model_single_beam(x_norm_plot_torch).cpu().numpy()[:, 0].flatten()


plt.figure(figsize=(8, 5))
plt.plot(x_phys_plot, w_analytic(x_phys_plot), 'r-', label='Analytic Solution')
plt.plot(x_phys_plot, w_pinn, 'b--', label='PINN Prediction')
plt.xlabel('x (m)')
plt.ylabel('Deflection $w(x)$ (m)')
plt.title('PINN vs Analytic Solution (Domain Decomposition)')
plt.legend()
plt.grid(True)
plt.show()


x_at_2m = 2.0
x_at_3m = 3.0
w_at_2m = w_analytic(np.array([x_at_2m]))[0] # [0] to get the scalar value from the array
w_at_3m = w_analytic(np.array([x_at_3m]))[0]

print(f"w_at_2m: {w_at_2m}, w_at_3m: {w_at_3m}")